Загружаем таблицу, содержащую выгруженные данные с ivi, а также размеченные тексты (85% от всех новостей). Предварительно тексты были отсортированы в случайном порядке, чтобы избежать возможных смещений. Описания фильмов размечались по принципу наличия в них указания на криминальный сюжет, где 1 - отсутствие криминала, 2 - его наличие. 

In [1]:
import pandas as pd
df = pd.read_excel ('выгрузка_разметка.xlsx', index_col = 0)
df.head()

,title,year,description,desc_bin,crime
1,Кошмар на курорте,2016,Лиза едет на море одна и сразу становится объе...,0,1.0
2,Без границ,2015,Фильм состоит из четырех новелл. Одна из них р...,1,1.0
3,Небесный суд,2012,"Андрей – прокурор первой ступени, а Вениамин –...",1,2.0
4,Битва за Севастополь,2015,Людмила Павличенко поступает на исторический ф...,1,1.0
5,Люблю тебя любую,2017,\nМолодые сестры Наталья и Лариса получают в н...,0,1.0


Смотрим на наполненность категорий и замечаем, что в наиболее важной для нас категории (2 - фильм содержит криминальный сюжет) наблюдений меньше, чем в другой, более чем в 2 раза. Такой разрыв может повлиять на результаты работы классификаторов, однако уменьшать количество наблюдений, закодированных единицей, не имеет особого смысла в связи с тем, что сама база имеет небольшой размер – содержит 577 описаний фильмов. 

In [2]:
df['crime'].value_counts()

1.0    360
2.0    130
Name: crime, dtype: int64

Выполняем предобработку данных: удаляем небуквенные символы, приводим слова к одному регистру, удаляем двойные пробелы, латинские символы (вроде их и не было, но на всякий случай) и лемматизируем.

In [3]:
def delete_punctuation(text):
    new_text = []
    for i in text:
        if i.isalpha() or i == ' ':
            new_text.append(i)
        else:
            new_text.append(" ")
    return ''.join(new_text)

In [4]:
df['description_prep'] = df['description'].apply(delete_punctuation)
df.loc[4, 'description_prep']

'Людмила Павличенко поступает на исторический факультет Киевского университета в      году  С друзьями в выходные она идет в стрелковый клуб и  впервые взяв в руки винтовку  выбивает    из     Ее вызывают к ректору и объявляют об обязательном направлении на   месячные курсы мастеров точного выстрела  В июне      Люда едет на практику в архив исторического музея в Одессу  Там она знакомится с молодым доктором Борисом  который в нее влюбляется  С началом мобилизации Людмила попадает в стрелковый отряд  Командир Вадим Жук учит своих стрелков подбивать танки бронебойными патронами  Отряд бросают на оборону Севастополя  После ранения Людмилу хотят комиссовать  но она требует у влюбленного в нее Бориса подписать рапорт о полном выздоровлении  чтобы вернуться на передовую  После гибели Жука командовать отрядом назначен Леонид Киценко  с которым у Людмилы случается роман  Возвращаясь с операции  Леонид и Людмила попадают в ловушку на минном поле  и он погибает  закрыв девушку своим телом  Немц

In [5]:
df['description_prep'] = df['description_prep'].str.lower()
df.loc[4, 'description_prep']

'людмила павличенко поступает на исторический факультет киевского университета в      году  с друзьями в выходные она идет в стрелковый клуб и  впервые взяв в руки винтовку  выбивает    из     ее вызывают к ректору и объявляют об обязательном направлении на   месячные курсы мастеров точного выстрела  в июне      люда едет на практику в архив исторического музея в одессу  там она знакомится с молодым доктором борисом  который в нее влюбляется  с началом мобилизации людмила попадает в стрелковый отряд  командир вадим жук учит своих стрелков подбивать танки бронебойными патронами  отряд бросают на оборону севастополя  после ранения людмилу хотят комиссовать  но она требует у влюбленного в нее бориса подписать рапорт о полном выздоровлении  чтобы вернуться на передовую  после гибели жука командовать отрядом назначен леонид киценко  с которым у людмилы случается роман  возвращаясь с операции  леонид и людмила попадают в ловушку на минном поле  и он погибает  закрыв девушку своим телом  немц

In [6]:
df

,title,year,description,desc_bin,crime,description_prep
1,Кошмар на курорте,2016,Лиза едет на море одна и сразу становится объе...,0,1.0,лиза едет на море одна и сразу становится объе...
2,Без границ,2015,Фильм состоит из четырех новелл. Одна из них р...,1,1.0,фильм состоит из четырех новелл одна из них р...
3,Небесный суд,2012,"Андрей – прокурор первой ступени, а Вениамин –...",1,2.0,андрей прокурор первой ступени а вениамин ...
4,Битва за Севастополь,2015,Людмила Павличенко поступает на исторический ф...,1,1.0,людмила павличенко поступает на исторический ф...
5,Люблю тебя любую,2017,\nМолодые сестры Наталья и Лариса получают в н...,0,1.0,молодые сестры наталья и лариса получают в на...
...,...,...,...,...,...,...
573,Ненавижу,2016,\nУ Ани образцовая семья: любящий и любимый му...,0,NaN,у ани образцовая семья любящий и любимый муж...
574,Под прицелом любви,2012,С крыши торгового центра упала Анна Власова – ...,0,NaN,с крыши торгового центра упала анна власова ...
575,Вопреки всему,2014,"играет молодую девушку Наташу, жизнь которой ...",0,NaN,играет молодую девушку наташу жизнь которой ...
576,Ирония удачи,2010,\n\nНе случайно Глеб Денежкин получил прозвище...,0,NaN,не случайно глеб денежкин получил прозвище ...


In [7]:
def delete_double_space(double_spaced_text):
    while '  ' in double_spaced_text:
        double_spaced_text = double_spaced_text.replace('  ', ' ')
    return double_spaced_text

In [8]:
df['description_prep'] = df['description_prep'].apply(delete_double_space)
df.loc[4, 'description_prep']

'людмила павличенко поступает на исторический факультет киевского университета в году с друзьями в выходные она идет в стрелковый клуб и впервые взяв в руки винтовку выбивает из ее вызывают к ректору и объявляют об обязательном направлении на месячные курсы мастеров точного выстрела в июне люда едет на практику в архив исторического музея в одессу там она знакомится с молодым доктором борисом который в нее влюбляется с началом мобилизации людмила попадает в стрелковый отряд командир вадим жук учит своих стрелков подбивать танки бронебойными патронами отряд бросают на оборону севастополя после ранения людмилу хотят комиссовать но она требует у влюбленного в нее бориса подписать рапорт о полном выздоровлении чтобы вернуться на передовую после гибели жука командовать отрядом назначен леонид киценко с которым у людмилы случается роман возвращаясь с операции леонид и людмила попадают в ловушку на минном поле и он погибает закрыв девушку своим телом немцы выпускают листовку о гибели советско

In [9]:
import re
def delete_latin(text):    
    new_text = []
    for symbol in str(text):
        if symbol not in re.findall(r'[A-Za-z]', str(text)):
            new_text.append(symbol)
    return ''.join(new_text)

In [10]:
df['description_prep'] = df['description_prep'].apply(delete_latin)

In [11]:
import pymystem3
mstem = pymystem3.Mystem()

In [12]:
def lemmas(not_lemmatized_text):
    return ''.join(mstem.lemmatize(not_lemmatized_text)).strip()

In [13]:
%%time

df['description_prep'] = df['description_prep'].apply(lemmas)
df

Wall time: 11min 17s


,title,year,description,desc_bin,crime,description_prep
1,Кошмар на курорте,2016,Лиза едет на море одна и сразу становится объе...,0,1.0,лиза ехать на море один и сразу становиться об...
2,Без границ,2015,Фильм состоит из четырех новелл. Одна из них р...,1,1.0,фильм состоять из четыре новелла один из они р...
3,Небесный суд,2012,"Андрей – прокурор первой ступени, а Вениамин –...",1,2.0,андрей прокурор первый ступень а вениамин адво...
4,Битва за Севастополь,2015,Людмила Павличенко поступает на исторический ф...,1,1.0,людмила павличенко поступать на исторический ф...
5,Люблю тебя любую,2017,\nМолодые сестры Наталья и Лариса получают в н...,0,1.0,молодой сестра наталья и лариса получать в нас...
...,...,...,...,...,...,...
573,Ненавижу,2016,\nУ Ани образцовая семья: любящий и любимый му...,0,NaN,у аня образцовый семья любить и любимый муж пр...
574,Под прицелом любви,2012,С крыши торгового центра упала Анна Власова – ...,0,NaN,с крыша торговый центр упасть анна власов дочь...
575,Вопреки всему,2014,"играет молодую девушку Наташу, жизнь которой ...",0,NaN,играть молодой девушка наташа жизнь который мо...
576,Ирония удачи,2010,\n\nНе случайно Глеб Денежкин получил прозвище...,0,NaN,не случайно глеб денежкин получать прозвище ве...


Создаем матрицу токенов и выставляем ограничение в 1%, чтобы убрать редкие слова. Несмотря на то, что матрицу в большинстве случаев стоит пытаться привести к квадратному виду, такой подход в контексте решения нашей задачи сильно снизил показатели качества/предсказательной силы моделей – вероятно, это связано с тем, что наблюдений сравнительно немного, а количество слов в нем достаточно велико. Именно поэтому было принято решение выбрать ограничение не в 4-5%, а в 1%. 

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(min_df = 0.01).fit(df['description_prep'])
matrix = cvect.transform(df['description_prep'])
matrix = pd.DataFrame(matrix.toarray(), index = df.index, columns = cvect.get_feature_names())
matrix

,аборт,абсолютно,авантюра,авария,автокатастрофа,автомобиль,автомобильный,агент,агентство,адвокат,...,юноша,юный,юра,юрий,являться,явно,язык,якобы,яркий,ярость
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
574,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
575,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
576,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Добавляем к получившейся матрице слов столбец с кодированием

In [15]:
matrix = pd.concat([df['crime'], matrix], axis = 1)
matrix

,crime,аборт,абсолютно,авантюра,авария,автокатастрофа,автомобиль,автомобильный,агент,агентство,...,юноша,юный,юра,юрий,являться,явно,язык,якобы,яркий,ярость
1,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,NaN,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
574,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
575,NaN,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
576,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Делим на 2 таблицы: первая таблица содержит размеченные новости, на которых будет проходить обучение и тестрирование, вторая – неразмеченные. 

In [16]:
df_known = matrix[matrix['crime'].notna()].copy()
df_unknown = matrix[matrix['crime'].isna()].copy()
display(df_known, df_unknown)

,crime,аборт,абсолютно,авантюра,авария,автокатастрофа,автомобиль,автомобильный,агент,агентство,...,юноша,юный,юра,юрий,являться,явно,язык,якобы,яркий,ярость
1,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
487,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
488,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489,1.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,crime,аборт,абсолютно,авантюра,авария,автокатастрофа,автомобиль,автомобильный,агент,агентство,...,юноша,юный,юра,юрий,являться,явно,язык,якобы,яркий,ярость
491,NaN,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
492,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
494,NaN,0,0,0,0,0,0,0,0,0,...,2,1,0,0,0,0,0,0,0,0
495,NaN,2,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,NaN,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
574,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
575,NaN,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
576,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Создаем 2 новых объекта: 1 – для зависимой переменной, 2 – для независимой (токены из матрицы). Выводим токены с шагом 50 для предварительно ознакомления.

In [17]:
dep_variable_bin = 'crime'
indep_variables = list(df_known.columns)[2:]
indep_variables[::50]

['абсолютно',
 'беседа',
 'бывший',
 'взаимный',
 'возвращаться',
 'встреча',
 'выступать',
 'давний',
 'для',
 'драка',
 'за',
 'занятие',
 'иван',
 'исключительно',
 'ключ',
 'кредит',
 'лишний',
 'мечтать',
 'набирать',
 'настя',
 'несколько',
 'обещание',
 'одноклассник',
 'от',
 'охрана',
 'пешком',
 'подписывать',
 'поле',
 'посещать',
 'предпочитать',
 'приковывать',
 'происшествие',
 'рабочий',
 'реальность',
 'рыдать',
 'семья',
 'случайность',
 'соперник',
 'ссора',
 'супруга',
 'темный',
 'убеждать',
 'университет',
 'финальный',
 'чемодан',
 'якобы']

Случайным образом делим таблицу с размеченными новостями так, чтобы первая подвыборка, на которой будет обучаться модель, составляла 70%, а подвыборка для тестирования – 30% соответственно.

In [18]:
import numpy as np
np.random.seed(20)
test_data_idx = np.random.choice(df_known.index, round(len(df_known)*0.3), replace = False)
test_data = df_known.loc[test_data_idx].copy()
train_data = df_known.drop(test_data_idx).copy()
display(df_known, test_data, train_data)

,crime,аборт,абсолютно,авантюра,авария,автокатастрофа,автомобиль,автомобильный,агент,агентство,...,юноша,юный,юра,юрий,являться,явно,язык,якобы,яркий,ярость
1,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
487,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
488,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489,1.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,crime,аборт,абсолютно,авантюра,авария,автокатастрофа,автомобиль,автомобильный,агент,агентство,...,юноша,юный,юра,юрий,являться,явно,язык,якобы,яркий,ярость
157,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
210,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
277,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
352,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
346,2.0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
201,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
467,1.0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
412,1.0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


,crime,аборт,абсолютно,авантюра,авария,автокатастрофа,автомобиль,автомобильный,агент,агентство,...,юноша,юный,юра,юрий,являться,явно,язык,якобы,яркий,ярость
1,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,2.0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
486,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
487,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489,1.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Переходим к построению предсказательных моделей. Сначала строим самое глубокое дерево с минимальным числом наблюдений в узлах.

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

classifier = DecisionTreeClassifier(min_samples_leaf = 1, 
                                   min_samples_split = 2, 
                                   random_state = 0)
classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int')) 

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

Смотрим на кол-во узлов и глубину.

In [20]:
classifier.tree_.node_count, classifier.tree_.max_depth 

(71, 19)

Сохряняем предсказания для обучающей и тестовой подвыборок.

In [21]:
predictions_bin_train = classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

display(predictions_bin_train, predictions_bin_test)

1      1
2      1
4      1
5      1
6      2
      ..
484    2
486    1
487    2
489    1
490    1
Name: Predictions, Length: 343, dtype: int32

157    1
210    1
277    1
352    1
346    2
      ..
401    1
201    2
467    2
412    1
46     1
Name: Predictions, Length: 147, dtype: int32

Переходим к оценке результатов предсказания модели. В первую очередь, обращаем внимание на категорию, которая является ненаполненной и которая нас интересует больше всего – 2 (криминальный сюжет). В данном случае можно заключить, что модель является переобученной: она не ошибается на обучающей подвыборке, но не очень хорошо предсказывает на тестовой, особенно это касается ненаполненной категории. Хотя, в целом, 57% – нормальный результат. Далее мы попробуем повысить качество модели.


In [22]:
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       254
           2       1.00      1.00      1.00        89

    accuracy                           1.00       343
   macro avg       1.00      1.00      1.00       343
weighted avg       1.00      1.00      1.00       343

              precision    recall  f1-score   support

           1       0.82      0.89      0.85       106
           2       0.64      0.51      0.57        41

    accuracy                           0.78       147
   macro avg       0.73      0.70      0.71       147
weighted avg       0.77      0.78      0.77       147



Выводим слова, которые позволяют лучше всего предсказывать категории. Половина из них, действительно, в нашем случае имеет достаточно высокую смысловую нагрузку, однако же в данном списке есть и те слова, которые, по логике, не должны вносить существенный вклад в категоризацию, например: его, любимый, имя и тд. Кроме того, нам не понравилось наличие в выдаче имени Егор, поэтому в дальнейшем мы планируем улучшить модель))

In [23]:
feat_imp = classifier.feature_importances_
feat_imp = pd.Series(feat_imp, index = indep_variables)
feat_imp.sort_values(ascending = False)[:50]

тюрьма           0.157542
преступник       0.087554
его              0.078287
расследование    0.058713
криминальный     0.048589
следователь      0.037837
вынуждать        0.036415
любимый          0.034214
способ           0.029359
отбирать         0.027484
жертва           0.026992
покрывать        0.026387
сюрприз          0.025895
сознание         0.025543
игорь            0.021577
имя              0.020231
бандит           0.018099
старший          0.017935
афера            0.017618
отец             0.016184
украсть          0.014962
загородный       0.014823
вешать           0.014686
переходить       0.014566
официантка       0.014552
будто            0.014513
алла             0.014419
зло              0.014288
егор             0.012644
бросать          0.011380
сделать          0.011380
переставать      0.010115
быть             0.010115
жить             0.007586
менее            0.007516
максим           0.000000
красота          0.000000
кофе             0.000000
кошмар      

Сохраняем вероятности текстов попасть в ту или иную категорию на проверочной подвыборке. Выводим кол-во размеченных, на основании алгоритма, описаний фильмов, содержащих криминальный сюжет. Далее выводим первые 5 описаний фильмов, относящихся к данной категории. 1-3 тексты, действительно, содержат явные указания на какой-либо криминал (тюрьма, убить, злоумышленники, покушения). 4 текст содержит слово "тюрьма", однако в данном случае оно является аллегорией. 5 текст включает упоминание проституции и слово "раскаяние", однако прямых указаний на криминал здесь также нет.

In [24]:
probabilities1 = classifier.predict_proba(df_unknown[indep_variables])
probabilities1 = pd.DataFrame(probabilities1, index = df_unknown.index, columns = ['ДеревоГл1', 'ДеревоГл2'])
probabilities1

,ДеревоГл1,ДеревоГл2
491,1.0,0.0
492,1.0,0.0
493,1.0,0.0
494,0.0,1.0
495,1.0,0.0
...,...,...
573,0.0,1.0
574,0.0,1.0
575,1.0,0.0
576,1.0,0.0


In [25]:
probabilities1['ДеревоГл2'].sum()

19.0

In [26]:
idx_high_proba = probabilities1['ДеревоГл2'].sort_values(ascending = False)[:5].index
for text in df.loc[idx_high_proba, 'description']:
    print(text)
    print('-----')

 классическом любовном треугольнике, развернувшемся в не самой романтичной и спокойной обстановке. На сей раз за любовь прекрасной женщины будут бороться мужчины с очень разными судьбами и характерами. 
Преуспевающий ландшафтный дизайнер Марина оказывается в незавидном положении. День знакомства жениха с ее родителями на носу, а она некстати поссорилась с любимым и очутилась на улице. Дверь захлопнулась на замок, но ключей у Марины нет. На помощь растерянной женщине приходит случайный знакомый Богдан, бывший военный офицер и человек со сложной судьбой. Как выясняется позже, Богдан только что вышел из тюрьмы, у него нет работы и ему некуда идти. Марину осеняет авантюрная идея: а что, если представить родителям вместо настоящего жениха своего нового приятеля? 
-----


Лера работает в кафе и варит лучший кофе в округе. У нее размеренная скромная жизнь, подруга, которую она вечно покрывает, и учеба на заочном. Но Лера не жалуется и свою работу очень любит. Поклонником ее таланта баристы ст

Нами были построены деревья с помощью самостоятельного подбора параметров. Лучшим оказалось дерево с максимальной глубиной 15 и минимальным кол-вом наблюдений в детском узле, равном 3. Здесь f1 остался таким же, однако немного увеличились средние значения по модели. Стоит сказать, что модель всё еще остается переобученной, однако данный вариант является самым оптимальным – при других параметрах качество стремительно падает. Выгружаем 5 текстов, для которых вероятность отнесения ко 2 категории наивысшая, и видим, что 1, 4 и 5 совпадают с выдачей по самому глубокому дереву. 2 описание фильма косвенно указывает на криминальный сюжет, а вот насчёт 3 описания алгоритм ошибся. 

In [46]:
classifier = DecisionTreeClassifier(min_samples_leaf = 3,
                                   max_depth = 15,
                                   random_state = 0)
classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int')) 

predictions_bin_train = classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))

probabilities2 = classifier.predict_proba(df_unknown[indep_variables])
probabilities2 = pd.DataFrame(probabilities2, index = df_unknown.index, columns = ['Дерево1', 'Дерево2'])

idx_high_proba = probabilities2['Дерево2'].sort_values(ascending = False)[:5].index
for text in df.loc[idx_high_proba, 'description']:
    print(text)
    print('-----')

train data
              precision    recall  f1-score   support

           1       0.96      0.97      0.97       254
           2       0.91      0.90      0.90        89

    accuracy                           0.95       343
   macro avg       0.94      0.93      0.94       343
weighted avg       0.95      0.95      0.95       343

test data
              precision    recall  f1-score   support

           1       0.82      0.92      0.87       106
           2       0.69      0.49      0.57        41

    accuracy                           0.80       147
   macro avg       0.76      0.70      0.72       147
weighted avg       0.79      0.80      0.78       147

 классическом любовном треугольнике, развернувшемся в не самой романтичной и спокойной обстановке. На сей раз за любовь прекрасной женщины будут бороться мужчины с очень разными судьбами и характерами. 
Преуспевающий ландшафтный дизайнер Марина оказывается в незавидном положении. День знакомства жениха с ее родителями на но

Смотрим на наиболее важные токены. Следует отметить, что их стало меньше, но первые 11 остались такими же. Интересно, что это за Игорь, который является даже важнее, чем слова "полиция" и "бандит".

In [28]:
feat_imp = classifier.feature_importances_
feat_imp = pd.Series(feat_imp, index = indep_variables)
feat_imp.sort_values(ascending = False)[:30]

тюрьма           0.189202
преступник       0.105149
его              0.094020
расследование    0.070512
криминальный     0.058354
следователь      0.045441
вынуждать        0.043733
любимый          0.041089
способ           0.035260
отбирать         0.033007
жертва           0.032416
несмотря         0.027333
путь             0.026240
игорь            0.025913
полиция          0.021463
бандит           0.020901
ум               0.020601
опасный          0.020360
магазин          0.018981
имущество        0.016583
шея              0.015988
рука             0.013884
конец            0.009407
дача             0.005813
ли               0.005315
происходить      0.003037
краска           0.000000
красавец         0.000000
красивый         0.000000
красавица        0.000000
dtype: float64

Чтобы подобрать лучшую модель, мы воспользовались кросс-валидацией, однако полученные в результате парметры оказались хуже, чем в нашей предыдущей модели. Число важных слов кардинально сократилось. По всей видимости, модель не переобучена, но плохо предсказывает необходимую нам категорию, поэтому это нам нам не подходит.

In [29]:
from sklearn.model_selection import GridSearchCV

In [30]:
%%time

params = {'max_depth': list(range(1,30)),
         'min_samples_leaf': list(range(1,40))}

score = 'f1_macro'
model = DecisionTreeClassifier(random_state = 0)

cv = GridSearchCV(model, params, score, cv = 4)
cv.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

Wall time: 2min 11s


GridSearchCV(cv=4, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=0, splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15

In [31]:
cv.best_params_,cv.best_score_

({'max_depth': 4, 'min_samples_leaf': 14}, 0.7251128504064079)

In [32]:
classifier = DecisionTreeClassifier(min_samples_leaf = 14,
                                   max_depth = 4,
                                   random_state = 0)
classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int')) 

predictions_bin_train = classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))


probabilities3 = classifier.predict_proba(df_unknown[indep_variables])
probabilities3 = pd.DataFrame(probabilities3, index = df_unknown.index, columns = ['ДеревоКВ1', 'ДеревоКВ2'])

idx_high_proba = probabilities3['ДеревоКВ2'].sort_values(ascending = False)[:5].index
for text in df.loc[idx_high_proba, 'description']:
    print(text)
    print('-----')

train data
              precision    recall  f1-score   support

           1       0.85      0.95      0.90       254
           2       0.79      0.54      0.64        89

    accuracy                           0.84       343
   macro avg       0.82      0.74      0.77       343
weighted avg       0.84      0.84      0.83       343

test data
              precision    recall  f1-score   support

           1       0.80      0.96      0.87       106
           2       0.79      0.37      0.50        41

    accuracy                           0.80       147
   macro avg       0.79      0.66      0.69       147
weighted avg       0.79      0.80      0.77       147

 классическом любовном треугольнике, развернувшемся в не самой романтичной и спокойной обстановке. На сей раз за любовь прекрасной женщины будут бороться мужчины с очень разными судьбами и характерами. 
Преуспевающий ландшафтный дизайнер Марина оказывается в незавидном положении. День знакомства жениха с ее родителями на но

In [33]:
feat_imp = classifier.feature_importances_
feat_imp = pd.Series(feat_imp, index = indep_variables)
feat_imp.sort_values(ascending = False)[:7]

тюрьма         0.452151
полиция        0.240054
его            0.142981
девушка        0.092763
происходить    0.072051
ярость         0.000000
красотка       0.000000
dtype: float64

Переходим к построению ансамбля деревьев. Сначала реализуем модель случайного леса. Модель с настройками по умолчанию уступает всем предыдущим (с т.з. предсказания категории 2). Мы попробовали поиграть с параметрами, но сильно улучшить модель не получилось. Однако большинство значимых для категоризации слов, выявленных посредством случайного леса (2 модель с настроенными параметрами), больше соотносятся со здравым смыслом, чем во всех предыдущих моделях.

In [34]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [35]:
rf_classifier = RandomForestClassifier(n_estimators=100,
                                      max_depth=None,
                                      random_state=0)
rf_classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

predictions_bin_train = rf_classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = rf_classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))

train data
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       254
           2       1.00      1.00      1.00        89

    accuracy                           1.00       343
   macro avg       1.00      1.00      1.00       343
weighted avg       1.00      1.00      1.00       343

test data
              precision    recall  f1-score   support

           1       0.78      0.97      0.87       106
           2       0.80      0.29      0.43        41

    accuracy                           0.78       147
   macro avg       0.79      0.63      0.65       147
weighted avg       0.79      0.78      0.74       147



In [36]:
rf_classifier = RandomForestClassifier(n_estimators=50,
                                      max_depth=20,
                                      random_state=0)
rf_classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

predictions_bin_train = rf_classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = rf_classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))


probabilities4 = rf_classifier.predict_proba(df_unknown[indep_variables])
probabilities4 = pd.DataFrame(probabilities4, index = df_unknown.index, columns = ['Лес1', 'Лес2'])

idx_high_proba = probabilities4['Лес2'].sort_values(ascending = False)[:5].index
for text in df.loc[idx_high_proba, 'description']:
    print(text)
    print('-----')

train data
              precision    recall  f1-score   support

           1       0.98      1.00      0.99       254
           2       1.00      0.96      0.98        89

    accuracy                           0.99       343
   macro avg       0.99      0.98      0.98       343
weighted avg       0.99      0.99      0.99       343

test data
              precision    recall  f1-score   support

           1       0.79      0.97      0.87       106
           2       0.82      0.34      0.48        41

    accuracy                           0.80       147
   macro avg       0.81      0.66      0.68       147
weighted avg       0.80      0.80      0.76       147

Нина – талантливый кондитер. Она печет изумительные свадебные торты. Она привозит торт своей подруге Свете, которую когда-то познакомила с ее будущим мужем Пашей. Нина везла торт на велосипеде по набережной и по дороге чуть не сбила мальчика Мишу, чем вызвала недовольство его отца Кости. Тем временем Костя обсуждает со свои

In [37]:
feat_imp = rf_classifier.feature_importances_
feat_imp = pd.Series(feat_imp, index = indep_variables)
feat_imp.sort_values(ascending = False)[:50]

тюрьма           0.025954
обвинять         0.017004
бандит           0.015225
расследование    0.014360
преступник       0.012505
полиция          0.011161
преступление     0.010610
загородный       0.010497
виновный         0.009191
следователь      0.008828
отомщать         0.008827
милиция          0.008435
убийство         0.008045
покушение        0.006707
арестовывать     0.006554
выяснять         0.005944
кража            0.005744
адвокат          0.005407
жертва           0.005277
мы               0.005216
не               0.005181
избавляться      0.005115
убивать          0.005025
на               0.004906
закон            0.004777
он               0.004389
свой             0.004353
выходить         0.004012
дочь             0.003991
подстраивать     0.003830
деньги           0.003829
но               0.003788
вор              0.003774
становиться      0.003703
вина             0.003628
она              0.003535
орган            0.003531
роман            0.003502
любить      

Переходим ко второму типу ансамблей. Реализуем бустинг. Результат подбора параметров вручную является наилучшим: даже несмотря на переобученность, она лучше всего предсказывает важную для нас и менее наполненную категорию (3 модель, f1 = 0.66). Тексты, размеченные как обладающие указанием на криминал, в проверочной подвыборке, действительно, таковыми и являются. Автоматический поиск оптимальных настроек для бустинга не увенчался успехом, хотя на него было потрачено достаточно большое количество времени в связи с установлением небольшого шага. 

In [38]:
gb_classifier = GradientBoostingClassifier(n_estimators=100,
                                      max_depth=None,
                                      random_state=0)
gb_classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

predictions_bin_train = gb_classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = gb_classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))

train data
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       254
           2       1.00      1.00      1.00        89

    accuracy                           1.00       343
   macro avg       1.00      1.00      1.00       343
weighted avg       1.00      1.00      1.00       343

test data
              precision    recall  f1-score   support

           1       0.83      0.91      0.86       106
           2       0.68      0.51      0.58        41

    accuracy                           0.80       147
   macro avg       0.75      0.71      0.72       147
weighted avg       0.79      0.80      0.79       147



In [39]:
gb_classifier = GradientBoostingClassifier(n_estimators=50,
                                      max_depth=10,
                                      random_state=0)
gb_classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

predictions_bin_train = gb_classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = gb_classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))

train data
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       254
           2       1.00      1.00      1.00        89

    accuracy                           1.00       343
   macro avg       1.00      1.00      1.00       343
weighted avg       1.00      1.00      1.00       343

test data
              precision    recall  f1-score   support

           1       0.83      0.94      0.88       106
           2       0.78      0.51      0.62        41

    accuracy                           0.82       147
   macro avg       0.81      0.73      0.75       147
weighted avg       0.82      0.82      0.81       147



In [40]:
gb_classifier = GradientBoostingClassifier(n_estimators=75,
                                      max_depth=10,
                                      random_state=0)
gb_classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

predictions_bin_train = gb_classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = gb_classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))


probabilities5 = gb_classifier.predict_proba(df_unknown[indep_variables])
probabilities5 = pd.DataFrame(probabilities5, index = df_unknown.index, columns = ['Бустинг1', 'Бустинг2'])

idx_high_proba = probabilities5['Бустинг2'].sort_values(ascending = False)[:5].index
for text in df.loc[idx_high_proba, 'description']:
    print(text)
    print('-----')


train data
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       254
           2       1.00      1.00      1.00        89

    accuracy                           1.00       343
   macro avg       1.00      1.00      1.00       343
weighted avg       1.00      1.00      1.00       343

test data
              precision    recall  f1-score   support

           1       0.85      0.94      0.89       106
           2       0.79      0.56      0.66        41

    accuracy                           0.84       147
   macro avg       0.82      0.75      0.78       147
weighted avg       0.83      0.84      0.83       147

Катя и Сергей - счастливая семейная пара. У них совместный бизнес. Джип последней модели. Квартира в современном доме в центре города. Всё настолько хорошо и благополучно, что супруги уже подумывают о загородном трёхэтажном коттедже с бассейном. Но однажды всё изменилось - Сергей исчез. Катя начинает собственное расследован

In [41]:
feat_imp = gb_classifier.feature_importances_
feat_imp = pd.Series(feat_imp, index = indep_variables)
feat_imp.sort_values(ascending = False)[:50]

тюрьма            0.149541
преступник        0.060896
его               0.060653
расследование     0.058340
бандит            0.038229
криминальный      0.037576
следователь       0.035386
полиция           0.028129
покрывать         0.025190
вынуждать         0.022978
любимый           0.021580
отбирать          0.020517
жертва            0.018429
совместный        0.015009
способ            0.014775
афера             0.013594
домик             0.013148
загородный        0.011261
ум                0.011179
старший           0.011110
угрожать          0.010605
юля               0.010021
игорь             0.009211
убийца            0.008399
алла              0.008361
исчезать          0.008191
бизнесмен         0.006293
она               0.006057
находить          0.005538
над               0.005476
на                0.005247
девушка           0.005239
шея               0.005070
выясняться        0.005030
правда            0.004753
сознание          0.004744
сюрприз           0.004741
в

In [147]:
import warnings
warnings.filterwarnings('ignore')

In [148]:
%%time

params = {'max_depth': list(range(7,25)),
         'n_estimators': list(range(30,130,5))}

score = 'f1'
model = GradientBoostingClassifier(random_state = 0)

cv = GridSearchCV(model, params, score, cv = 4)
cv.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

Wall time: 1h 7min 59s


GridSearchCV(cv=4, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_c...
                 

In [149]:
cv.best_params_,cv.best_score_

({'max_depth': 8, 'n_estimators': 35}, 0.8758369568427533)

In [42]:
gb_classifier = GradientBoostingClassifier(n_estimators=35,
                                      max_depth=8,
                                      random_state=0)
gb_classifier.fit(train_data[indep_variables], train_data[dep_variable_bin].astype('int'))

predictions_bin_train = gb_classifier.predict(train_data[indep_variables])
predictions_bin_train = pd.Series(predictions_bin_train,
                                 index = train_data[indep_variables].index,
                                 name = 'Predictions')

predictions_bin_test = gb_classifier.predict(test_data[indep_variables])
predictions_bin_test = pd.Series(predictions_bin_test,
                                 index = test_data[indep_variables].index,
                                 name = 'Predictions')

print('train data')
print(classification_report(train_data[dep_variable_bin].astype('int'), predictions_bin_train))

print('test data')
print(classification_report(test_data[dep_variable_bin].astype('int'), predictions_bin_test))



train data
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       254
           2       1.00      1.00      1.00        89

    accuracy                           1.00       343
   macro avg       1.00      1.00      1.00       343
weighted avg       1.00      1.00      1.00       343

test data
              precision    recall  f1-score   support

           1       0.81      0.94      0.87       106
           2       0.74      0.41      0.53        41

    accuracy                           0.80       147
   macro avg       0.77      0.68      0.70       147
weighted avg       0.79      0.80      0.78       147



In [47]:
probabilities_full = pd.concat([probabilities1, probabilities2, probabilities3, probabilities4, probabilities5], axis = 1)
probabilities_full

,ДеревоГл1,ДеревоГл2,Дерево1,Дерево2,ДеревоКВ1,ДеревоКВ2,Лес1,Лес2,Бустинг1,Бустинг2
491,1.0,0.0,0.995122,0.004878,0.891304,0.108696,0.956663,0.043337,0.992401,0.007599
492,1.0,0.0,0.995122,0.004878,0.891304,0.108696,0.762508,0.237492,0.982403,0.017597
493,1.0,0.0,0.995122,0.004878,0.891304,0.108696,0.753937,0.246063,0.986935,0.013065
494,0.0,1.0,0.000000,1.000000,0.760000,0.240000,0.665863,0.334137,0.312087,0.687913
495,1.0,0.0,0.995122,0.004878,0.891304,0.108696,0.687871,0.312129,0.994749,0.005251
...,...,...,...,...,...,...,...,...,...,...
573,0.0,1.0,0.333333,0.666667,0.235294,0.764706,0.700642,0.299358,0.048095,0.951905
574,0.0,1.0,0.000000,1.000000,0.891304,0.108696,0.655699,0.344301,0.236616,0.763384
575,1.0,0.0,0.995122,0.004878,0.891304,0.108696,0.922143,0.077857,0.994749,0.005251
576,1.0,0.0,0.995122,0.004878,0.891304,0.108696,0.952626,0.047374,0.994749,0.005251


In [48]:
probabilities_full['Текст'] = df['description']
probabilities_full

,ДеревоГл1,ДеревоГл2,Дерево1,Дерево2,ДеревоКВ1,ДеревоКВ2,Лес1,Лес2,Бустинг1,Бустинг2,Текст
491,1.0,0.0,0.995122,0.004878,0.891304,0.108696,0.956663,0.043337,0.992401,0.007599,Юная Настя приводит домой репетитора. Но её ма...
492,1.0,0.0,0.995122,0.004878,0.891304,0.108696,0.762508,0.237492,0.982403,0.017597,Лиза мечтает побывать в Венеции. Вместе с ней ...
493,1.0,0.0,0.995122,0.004878,0.891304,0.108696,0.753937,0.246063,0.986935,0.013065,Татьяна вступила на непростой путь. Она хочет ...
494,0.0,1.0,0.000000,1.000000,0.760000,0.240000,0.665863,0.334137,0.312087,0.687913,"Фотографии манифестаций, которые требуют от Бр..."
495,1.0,0.0,0.995122,0.004878,0.891304,0.108696,0.687871,0.312129,0.994749,0.005251,Женя — юная девушка из провинции. Вопреки воле...
...,...,...,...,...,...,...,...,...,...,...,...
573,0.0,1.0,0.333333,0.666667,0.235294,0.764706,0.700642,0.299358,0.048095,0.951905,\nУ Ани образцовая семья: любящий и любимый му...
574,0.0,1.0,0.000000,1.000000,0.891304,0.108696,0.655699,0.344301,0.236616,0.763384,С крыши торгового центра упала Анна Власова – ...
575,1.0,0.0,0.995122,0.004878,0.891304,0.108696,0.922143,0.077857,0.994749,0.005251,"играет молодую девушку Наташу, жизнь которой ..."
576,1.0,0.0,0.995122,0.004878,0.891304,0.108696,0.952626,0.047374,0.994749,0.005251,\n\nНе случайно Глеб Денежкин получил прозвище...


В качестве лучших моделей у нас выступил бустинг (f1 = 0.66) и дерево (f1 = 0.57), поэтому сравним их, выведя 7 текстов, предсказания по которым отличаются сильнее всего.

In [56]:
probabilities_full['Вычисления2'] = abs(probabilities_full['Бустинг2']-probabilities_full['Дерево2'])
probabilities_full = probabilities_full.sort_values('Вычисления2', ascending=False)
probabilities_full[:7]

,ДеревоГл1,ДеревоГл2,Дерево1,Дерево2,ДеревоКВ1,ДеревоКВ2,Лес1,Лес2,Бустинг1,Бустинг2,Текст,Вычисления2
516,1.0,0.0,0.000000,1.000000,0.891304,0.108696,0.777932,0.222068,0.954067,0.045933,"Вадим Петрович Соколов возглавляет небольшую, ...",0.954067
529,0.0,1.0,0.000000,1.000000,0.760000,0.240000,0.854434,0.145566,0.924879,0.075121,Большой город. Рассвет. Молодой мужчина по име...,0.924879
500,0.0,1.0,0.995122,0.004878,0.891304,0.108696,0.859101,0.140899,0.172174,0.827826,\n\nЛера работает в кафе и варит лучший кофе в...,0.822948
555,0.0,1.0,0.000000,1.000000,0.315789,0.684211,0.908119,0.091881,0.784231,0.215769,Вере уже тридцать лет. Главная проблема в жизн...,0.784231
548,0.0,1.0,0.250000,0.750000,0.891304,0.108696,0.819493,0.180507,0.958833,0.041167,\nМаша никак не может смириться со смертью сое...,0.708833
536,0.0,1.0,0.250000,0.750000,0.315789,0.684211,0.874541,0.125459,0.936683,0.063317,"\n\nГлавный герой сериала, в прошлом большой л...",0.686683
513,0.0,1.0,0.995122,0.004878,0.891304,0.108696,0.759451,0.240549,0.308611,0.691389,счастливой семейной паре – Тине и Стасе Кирилл...,0.686511


В 1 случае дерево отнесло текст к категории 2, а бустинг – к категории 1. Результат бустинга правильный, тк в тексте нет указаний на наличие криминального сюжета.

In [50]:
probabilities_full['Текст'][516]

'Вадим Петрович Соколов возглавляет небольшую, но обладающую хорошей репутацией аудиторскую компанию. Дела идут вроде бы неплохо. Да и в личной жизни Соколову жаловаться не на что. Он удачно женат на Альбине  без ума от своей дочурки Катеньки. Но в последнее время бизнес почему-то начинает давать сбои. Анализируя ситуацию, Вадим Петрович приходит к выводу, что его секретарь-референт Маргарита  стала хуже работать. Подыскивая замену, Соколов знакомится с Надеждой С профессиональной точки зрения – кандидатура безупречная. Беда приходит, откуда ее не ждали. Новая сотрудница страстно влюбляется в шефа. Семейное счастье Вадима под угрозой, Альбина не намерена терпеть соперницу. Между тем, Надежда напролом устремляется к заветной цели. Ведь она уверена, Соколов – именно тот мужчина, который создан для нее. \n'

В этом случае дерево также отнесло текст к категории 2, а бустинг – к категории 1. Мы снова склонны согласиться с бустингом, тк явных указаний на криминал здесь нет.

In [52]:
probabilities_full['Текст'][529]

'Большой город. Рассвет. Молодой мужчина по имени Фердинанд сидит на пристани и пьёт газировку - это его любимая привычка. В это же время молодая девушка Ева, которая любит читать научные книги и записывать свои мысли в дневник, но при этом работает женщиной лёгкого поведения, ищет в городе клиентов. Неожиданно пути героев пересекаются...Мужчина приглашает Еву к себе домой. Они разговаривают и делятся событиями из жизни. Фердинанд рассказывает свою печальную историю о жене и о том, что она ему изменяет. Уличив её в проступке, он услышал раскаяние и обещание, что этого не повторится. Но тем не менее, недоверчивый муж нанял жиголо, который должен соблазнить его жену. И подставному любовнику это удаётся. Именно по этой причине главный герой и сидит сейчас в комнате с Евой. Та, в свою очередь, рассказывает случайному собеседнику о гибели мужа и свекрови Вере Павловне, которая нуждается в уходе из-за отнявшихся ног, но при этом всей душой ненавидит девушку. Два совершенно чужих человека про

Здесь дерево снова ошиблось и отнесло к 1 категории описание фильма, которое бустинг определил как обладающее криминальным сюжетом, и оказался прав.

In [53]:
probabilities_full['Текст'][500]

'\n\nЛера работает в кафе и варит лучший кофе в округе. У нее размеренная скромная жизнь, подруга, которую она вечно покрывает, и учеба на заочном. Но Лера не жалуется и свою работу очень любит. Поклонником ее таланта баристы становится постоянный клиент - мужчина, смутно кажущийся ей очень знакомым и родным. Со временем он признается, что он Лерин отчим, которого она много лет считала погибшим.\n\nОднако радость девушки длится недолго. Спустя несколько дней отчим умирает якобы от сердечного приступа, а Лере начинает казаться, что кто-то пытается убить и её.'

Несмотря на наличие смерти в описании фильма, явного криминального сюжета в нем нет, поэтому бустинг снова оказался прав, определив данное описание к 1 категории.

In [54]:
probabilities_full['Текст'][555]

'Вере уже тридцать лет. Главная проблема в жизни этой женщины – невозможность иметь детей. Без этого ее брак назвать счастливым невозможно. Ее муж Павел тоже озабочен этим обстоятельством. Он предпринимает решительные шаги для того, чтобы обзавестись наследниками. В этом ему помогает Яна, она становится любовницей Павла. Вера через несколько лет узнает о том, что ее муж практически живет на две семьи. Более того, Яна смогла дать Павлу то, чего он так и не дождался от Веры - у нее родился ребенок. Муж уходит от Веры. Но счастье Павла и его новой жены длится недолго. Они погибают в автомобильной катастрофе. Их ребенок остается круглый сиротой. Тогда Вера принимает нелегкое решение. Она хочет заменить ребенку любимого мужчины его родную мать. Сможет ли она решить эту непростую психологическую задачу, полюбить маленького человека, который стал причиной крушения ее личного счастья?\n'

Здесь ситуация неочевидна, потому как прямых указаний на криминальные действия нет, однако есть слово "месть" и описание желания лишить героиню наследства (вероятно, всё-таки незаконным путём). Мы более склонны в данном случае согласиться с деревом, определившим попадание текста во вторую категорию с вероятностью 0.75. 

In [55]:
probabilities_full['Текст'][548]

'\nМаша никак не может смириться со смертью соей матери, которая случилась, когда девочке было всего десять лет. Поэтому появление новой возлюбленной отца неизбежно приводит к постоянным ссорам. Когда отец в результате несчастного случая попадает в тяжелом состоянии в больницу, терпение Маши заканчивается и она уходит из дома. \n\nТем временем партнер отца по ресторанному бизнесу долгие годы вынашивает мысли о мести и собирается отобрать ресторан и лишить Машу наследства.\n'

В данном случае, опять же, неизвестно, будет ли какой-то криминальный сюжет в фильме, но считаем, что подкидывать детей – аморально) Наши мнения по вопросу отнесения описания данного фильма к той или иной категории разошлись, равно как результаты предсказания дерева и бустинга. 
Коллеги, а как вы думаете, к какой категории стоит отнести этот текст?

In [57]:
probabilities_full['Текст'][536]

'\n\nГлавный герой сериала, в прошлом большой ловелас Игорь, остепенился и собирается жениться. Но прямо накануне свадьбы его ждет сюрприз: ему подкинули корзину с младенцем. \n\nВ корзине была записка, в которой сообщалось, что этот ребенок – его сын, и потому именно он должен заботиться о малыше. Разумеется, невеста устраивает жуткий скандал и отменяет свадьбу. Но теперь Игорь не одинок. И ему не терпится узнать, кто же одарил его таким подарочком.\n\nПриступая к поискам мамы-кукушки, Игорь вспоминает всех женщин, с которыми у него были интрижки, и наведывается к ним в гости. Но каждая из них отрицает материнство… Так чей же все-таки ребенок маленький Андрюша?\n'

Здесь, на наш взгляд, проиграл бустинг. В данном описании не содержится указание на наличие криминального сюжета.

In [59]:
probabilities_full['Текст'][513]

'счастливой семейной паре – Тине и Стасе Кирилловых. Они молодые архитекторы, которые недавно приобрели уютный загородный дом. У семьи Кирилловых все хорошо складывается не только дома, но и на работе. Однако за их семьей постоянно наблюдает Марьяна – подруга Тины, которая влюбилась в Стаса еще в студенчестве, но он выбрал Тину, а не ее. Проходит время. Марьяна выходит замуж за Алексея, но чувства к Стасу не дают ей покоя. Завидуя счастью Тины и Стаса, Марьяна решается пойти на все, лишь бы разрушить их брак и вернуть себе возлюбленного. А в это же время разлад в семью Кирилловых решает внести няня, которая присматривает за их детьми.'

Таким образом, мы можем заключить, что бустинг показал себя лучше. Вероятно, модель является переобученной, однако в сравнении со всеми остальными, она показывает наилучший результат и обладает более высокой прогностической способностью.